# Решение задачи классификации текстов по сантименту при помощи нейронных сетей

### Эпиграф

Будучи глубоко неудовлетворен теми результатами, которых позволяла достичь линейная и логистическая регрессия, я решил отложить сдачу финального проекта до тех пор, пока не обучусь нейросетям. Т.к. в настоящей специализации они проходились, я считаю такое решение задачи нейросетями совершенно легитимным.


In [18]:
!pip install nltk
!pip install tensorflow
!pip install gensim

In [19]:
#1. Импортируем общие библиотеки.
import pandas as pd
import numpy as np
import nltk
nltk.download("punkt")
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rookie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
#2. Импортируем тензорфлоу и керас
#Рассчитано на TF 2.0^

import tensorflow as tf
print(tf.__version__)
import keras
import keras.backend as K
import keras.layers as L
import tensorflow.compat.v1 as v1
np.random.seed(42)

#Функция очистки сессии
def reset_tf_session():
    curr_session = v1.get_default_session()
    if curr_session is not None:
        curr_session.close()
    # reset graph
    K.clear_session()
    # create new session
    config = v1.ConfigProto()
    config.gpu_options.allow_growth = True
    s = v1.InteractiveSession(config=config)
    v1.keras.backend.set_session
    return s

reset_tf_session()

2.3.0


Идея состоит в следующем.
1. Т.к. данные довольно хорошо предобработаны (например, все приведено к нижнему регистру, пунктуация уже отделена пробелами), разбиваем исходные тексты на слова простым методом .split()
2. Делим тексты на биграммы, создаем словарь биграмм, который отображает биграмму в целое число (индекс словаря), причем при применении к тестовым данных в будущем не будем учитывать порядок слов в биграмме.
3. На основе предобученных векторизаций GloVe строим матрицу векторизации биграмм, причем векторизация биграммы будет усреднением векторизаций слов.
4. Подаем векторизованные биграммы из исходных текстов на вход нейронной сети и обучаем ее.

In [24]:
from nltk import bigrams, ngrams, everygrams
from sklearn.model_selection import train_test_split
from gensim.models import phrases, word2vec


In [25]:
#3. Готовим первичные выборки, разбиваем тексты на n-граммы от одного слова до четырех с помощью nltk.everygrams, делаем словарь
df = pd.read_csv("products_sentiment_train.tsv", sep='\t', header=None)
df.columns = ["text", "label"]

X = [x[0] for x in df[["text"]].values.tolist()]   #Выборка, где элемент - строка 
Y = [y[0] for y in df[["label"]].values.tolist()]

#Преобразует тексты в список токенов, где биграммы будут вида X_Y
def tokenize_texts(texts):
    return [list(everygrams(text.split(), 1, 2)) for text in texts]

#Готовим словарь
#Плоский список всех слов всех текстов
flat_list = [item for sublist in tokenize_texts(X) for item in sublist]
#Список уникальных слов
all_words = list(set(flat_list))

del flat_list
UNK = "#UNK" #Символ "неизвестное слово", будет кодироваться нулем или нулевым вектором

all_words = [UNK] + all_words
vocab = {word: idx for idx, word in enumerate(all_words)}

len(X), len(Y), len(vocab), (X[0], tokenize_texts(X[0:1]))

(2000,
 2000,
 25080,
 ('2 . take around 10,000 640x480 pictures .',
  [[('2',),
    ('.',),
    ('take',),
    ('around',),
    ('10,000',),
    ('640x480',),
    ('pictures',),
    ('.',),
    ('2', '.'),
    ('.', 'take'),
    ('take', 'around'),
    ('around', '10,000'),
    ('10,000', '640x480'),
    ('640x480', 'pictures'),
    ('pictures', '.')]]))

In [26]:
#4.1 Функции для работы модели
from keras.preprocessing.sequence import pad_sequences
#Размер батча
N_BATCH = 4

#Функция возвращает индекс из словаря для биграммы, неважен порядок слов в биграмме
def ngram(t):
    if t in vocab:
        return vocab[t]
    else:
        return vocab[UNK]

def any_of_ngram(t):
    if len(t) == 1:
        if t in vocab:
            return vocab[t]
        else:
            return vocab[UNK]
    if len(t) == 2:
        if (t[0], t[1]) in vocab:
            return vocab[t]
        elif (t[1], t[0]) in vocab:
            return vocab[(t[1], t[0])]
        else:
            return vocab[UNK]    

#Преобразуем биграммы в индексную последовательность
def to_sequence(tokens):
    return np.array([ngram(t) for t in tokens])

#Преобразуем список текстов в Numpy матрицу последовательностей формы (<колво текстов>, <длина самого длинного из них>). 
#Последовательности в матрице дополняем нулями справа до нужной длины. Строк полностью из нулей быть не должно.
#Пример
#Словарь: {'A': 1, 'B': 2, 'C': 3}
#Тексты:  
#[['A B B C'], ['B C A']]
#На выходе матрица 
#[[1, 2, 2, 3],
# [2, 3, 1, 0]]
def to_matrix(texts, maxlen=0): #- для варианта с СОБСТВЕННЫМ СЛОВАРЕМ
    seqs = [to_sequence(tokens) for tokens in tokenize_texts(texts)]
    if maxlen == 0:
        maxlen = min(9999, max(list(map(len, seqs))))
    return pad_sequences(seqs, maxlen=maxlen, dtype='int32', padding='post', truncating='post', value=0)

#Выбираем из исходных выборок батч заданной длины, X прогоняем через to_matrix, Y просто преобразуем к формату numpy
#Эта функция не применяется, т.к. достаточно памяти для загрузки всех данных сразу
def get_batch(xs, y, count=N_BATCH):
    offset = 0
    while True:
        x_batch = x[offset:offset+count]
        X_batch = to_matrix()
        LR_batch = np.array(pipeline.predict(x_batch))
        Y_batch = np.array(y[offset:offset+count])
        yield [X_batch, LR_batch], Y_batch
        offset += count
        if offset >= len(x)//count*count:
            offset = 0
            
to_matrix(X[0:3])

array([[14324, 12915, 18820, 23732,  9311, 16272, 18553, 12915, 20580,
         8331, 18623, 12540, 23731,  5991,  3242,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [11578, 14635, 23645,  3620,  9390,  2065, 23446,  3675,  7571,
        13095, 20179, 11974, 20179, 14787,  1121,  6094,  7903, 23645,
        23446,  7003,  6148,  4696, 17706, 18068, 12915, 10527, 19049,
        17745, 12178,  5145,  9952, 10050,  5222,  3783, 22879, 15122,
        11845,  9510,  8519, 24452,  4323,  8332, 20081,  2228, 18076,
        18555, 23094, 15288,  5520,     0,     0,     0,     0,     0,
            0],
       [ 8399,  8703, 11305,  8399, 15032, 12782, 23645,  5423,  6524,
        13339, 10448,  7738, 14558,  1093,  1

In [ ]:
#4.2
#Используем keras.layers.Embedding

In [54]:
#4.3. Строим эмбеддинг на основе word2vec для биграмм путем сложения предварительно рассчитанных коэффициентов для каждого слова из биграммы
EMBEDDING_DIM = 32

from gensim.models import Word2Vec

model = Word2Vec([x.split() for x in X], size=EMBEDDING_DIM, window=5, min_count=1, workers=4)

def get_embedding(word):
    if word in model:
        return model[word]
    else:
        #Если слово неизвестно GloVe -> нулевой вектор
        return np.zeros((1,EMBEDDING_DIM))

#Готовим матрицу коэффициентов
embedding_matrix = np.zeros((len(vocab), EMBEDDING_DIM))
for ngram, i in vocab.items():
    embedding_vector = np.zeros((1,EMBEDDING_DIM))
    for word in ngram:
        embedding_vector += get_embedding(word)
    embedding_matrix[i] = embedding_vector / len(ngram) 

#Готовый слой Керас
embedding_layer = L.Embedding(len(vocab),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix], 
                            trainable=False, 
                            mask_zero=True)

model[['I']]

C:\bin\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':
C:\bin\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\bin\anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


KeyError: "word 'I' not in vocabulary"

Перейдем к построению модели. Модель будет включать обученный на GloVe слой эмбеддинга, который будет транспонировать биграммы в пространство признаков Glove, далее в модель включим LSTM с 1024 нейронами, далее переходной полносвязный слой на 256 нейронов и выходной на 1 бинарный признак с функцией активации softmax.

In [46]:
#5.1. Модель 1 - дает точность 0.7666
N_EMBED = 32
N_LSTM = 1024
N_BATCH = 4

#Используем оптимизатор Adam и бинарную кроссэнтропию в качестве функции потерь
def build_model():
    #Input - входные данные, шейп (None, None) означает, что мы можем принимать батч произвольной длины 
    X = L.Input(batch_input_shape=(None,None))
    #LR = L.Input(batch_input_shape=(None,))
    #Embedding - обеспечивает выравнивание входных данных и компактное представление одного элемента последовательности в 16 числах
    e = L.Embedding(len(vocab), N_EMBED, mask_zero=True)(X)
    #e = embedding_layer(X)
    #LSTM - Long Short Term Memory - обеспечивает анализ последовательности
    l2 = L.LSTM(units=N_LSTM, return_sequences=False, name="l1", dropout=0.25)(e)
    #Выходной слой обеспечивает сворачивание в бинарное значение, активация на выходном слое сигмоиду для значения [0..1]
    d = L.Dense(256, activation="relu")(l2)
    d = L.Dropout(0.3)(d)
    Y = L.Dense(1, activation="sigmoid")(d)
    return keras.models.Model(inputs=(X), outputs=Y)

t_model = build_model()
opt = keras.optimizers.Adam(lr=0.001)
t_model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
t_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 32)          802560    
_________________________________________________________________
l1 (LSTM)                    (None, 1024)              4329472   
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 5,394,689
Trainable params: 5,394,689
Non-trainable params: 0
____________________________________________

In [45]:
#Функции для тренировки моделей
#до третьей эпохи шаг обучения 0.0001 затем начинает уменшаться
def _schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

#Вариант с генератором - мне не нужен, т.к. на моей видеокарте все умещается в видеопамять
def train_model_gen(xt, yt, initial_epoch=0, n_epochs=10):
    t_model.fit(get_batch(xt, yt), epochs=n_epochs, steps_per_epoch=len(xt)//N_BATCH,
                shuffle=True, initial_epoch=initial_epoch,  callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(filepath='./W3/model.{epoch:03d}.hdf5'),
                    tf.keras.callbacks.LearningRateScheduler(_schedule)
                ])

#Вариант с целиковой обработкой - уменшения шага обучения
def train_model(xt, yt, 
                #xv, yv,
                initial_epoch=0, n_epochs=10):
    t_model.fit(to_matrix(xt), np.array(yt), epochs=n_epochs, 
                shuffle=True,
                #validation_data=(xv, yv),
                batch_size=N_BATCH, initial_epoch=initial_epoch, callbacks=[
                    tf.keras.callbacks.LearningRateScheduler(_schedule),
                    #tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
                ])

In [49]:
reset_tf_session()
t_model.reset_states()

train = train_model

opt = keras.optimizers.Adam(lr=0.0001)
t_model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])


train_model(X, Y, 0, 5)
#for i in range(10):
    #Xt, Xv, Yt, Yv = train_test_split(X, Y, train_size=0.75)
   #train(Xt, Yt, Xv, Yv, initial_epoch=i, n_epochs=i+1)
 

Epoch 1/5
500/500 [==============================] - 17s 34ms/step - loss: 0.6563 - accuracy: 0.6370
Epoch 2/5
500/500 [==============================] - 17s 34ms/step - loss: 0.5897 - accuracy: 0.6855
Epoch 3/5
500/500 [==============================] - 17s 34ms/step - loss: 0.3077 - accuracy: 0.8800
Epoch 4/5
500/500 [==============================] - 17s 34ms/step - loss: 0.0960 - accuracy: 0.9700
Epoch 5/5
500/500 [==============================] - 17s 34ms/step - loss: 0.0317 - accuracy: 0.9900


In [50]:
df = pd.read_csv("products_sentiment_test.tsv", sep='\t')
X_test = [x[0] for x in df[["text"]].values.tolist()]

Y_pred = [ 0 if y < 0.5 else 1 for y in t_model.predict(to_matrix(X_test))]
len(X_test), len(Y_pred)

(500, 500)

После обучения в течение 50 эпох точность составила 1.0 на обучаемой выборке. Применим к тестовой выборке.

In [51]:
df = pd.DataFrame()
df["y"] = Y_pred
df.head()

df.to_csv("kaggle_submission.csv", sep=',', index_label="Id")

In [ ]:
from IPython.display import Image

Image(filename = 'screen.png')